In [ ]:
analysis <- "Cropseq_20_14"
organ <- "murine"

library(Seurat)
library(ggplot2)
library(gridExtra)
library(stringi)
library(grid)
library(gridExtra)
library(RColorBrewer)
library(pheatmap)
library(matrixStats)
library(ggrepel)
library("pheatmap")
library("RColorBrewer")

## Path to folders
root <- dirname(getwd())

data <- file.path(root, "data")
raw <- file.path(data, "raw")
RData <- file.path(data, "RData")
processed <- file.path(data, "processed")
pdf <- file.path(processed, "pdf")
csv <- file.path(processed, "csv")
interim <- file.path(data, "interim")
external <- file.path(data, "external")
notebooks <- file.path(root, "notebook")

sfile <- function(path, name){
    return(file.path(path, paste0(analysis, "_", organ, "__", name)))
}

In [ ]:
load.10x <- function(path, file.name=NULL, cellranger.version = 2){

    files <- c("genes.tsv", "barcodes.tsv", "matrix.mtx")
    folders <- c("outs", "filtered_gene_bc_matrices", "filtered_gene_bc_matrices_mex")

    if(cellranger.version == 3){
        files <- c("features.tsv", "barcodes.tsv", "matrix.mtx")
        files <- c(files, paste0(files, ".gz"))
        folders <- c("outs", "filtered_feature_bc_matrix", "filtered_feature_bc_matrix_mex")
    }
    if(!is.null(file.name))
        files <- c(file.name)
    require(stringr)
    if(str_sub(path, start = -1) == "/")
        path = str_sub(path, start=0, end=-2)

    if(!file.exists(path))
        stop(paste("Path does not exist:", path))

    current.files <- list.files(path)
    return_list = list()

    if(length(current.files) == 1 && !(current.files %in% folders)){
        cat(paste0("Genome used:  ", current.files[1], "\n"))
        return_list <- load.10x(paste0(path, "/", current.files[1]),  file.name=file.name, cellranger.version = cellranger.version)
    }else{
        for(dir in current.files){
            if(dir %in% paste0(files, ".gz") && cellranger.version == 3){
                message("Unzip ", path, "/", dir)
                system(paste0("gunzip ", path, "/", dir))
            }
        }
    current.files <- list.files(path)

    for(dir in current.files){
        if(dir %in% files){
                if(str_sub(dir, start=-3, end = -1) == "mtx"){
                    require(Matrix)
                    return_list[[dir]] <- readMM(paste0(path, "/", dir))
                    cat(paste0("File in list ", path, "/", dir, "\n"))
                }else{
                    return_list[[dir]] <- read.table(paste0(path, "/", dir))
                    cat(paste0("File in list ", path, "/", dir, "\n"))
                }
            }else{
                if(dir %in% folders)
                    return_list <- load.10x(path=paste0(path, "/", dir), file.name=file.name, cellranger.version = cellranger.version)
            }
        }
    }

    if(length(return_list) == 0)
        stop("Could not identify 10x Stucture")

    return(return_list)

}

In [ ]:
convertTo_cellgRNA_table <- function(matrix, threshold = 1){
    rows <- rownames(matrix)
    cols <- colnames(matrix)

    cell_gRNA_tab <- data.frame(cell_barcode =  cols, stringsAsFactors = FALSE)
    cell_gRNA_tab$gRNA <- sapply(cell_gRNA_tab$cell_barcode, function(x){
        if(!(TRUE %in% (matrix[,x] >= threshold))) NA
        else paste0(rows[matrix[,x] >= threshold], collapse ="|")
    })
    cell_gRNA_tab$ngRNA <- colSums(matrix >= threshold)

    return(cell_gRNA_tab)

}

In [ ]:
normalize_expression <- function(matrix, scaling=10000){
    norm_matrix <- matrix(0, nrow=nrow(matrix), ncol=ncol(matrix))
    rownames(norm_matrix) <- rownames(matrix)
    colnames(norm_matrix) <- colnames(matrix)

    lib_size <- colSums(matrix)
    norm_matrix <- log2(1 + t(t(matrix)/lib_size) * scaling)

    return(norm_matrix)
}

In [ ]:
# Load 10X data from raw data
files_10x <- load.10x("../data/raw/", cellranger.version = 3)
rownames(files_10x$matrix.mtx) <-  make.unique(as.vector(files_10x$features.tsv$V2))
colnames(files_10x$matrix.mtx) <- files_10x$barcodes.tsv$V1

In [ ]:
# Load annotation data which used for alignment
annotation <- read.csv("TH17_Signaturelibrary.csv", sep="\t")

In [ ]:
# Matrix of gRNA names
matrix_gRNA <- as.matrix(files_10x$matrix.mtx[paste0("gRNA_", annotation$oligo_name, "_gene"),])

# Matrix of gene expression with no gRNA
matrix_regular <- as.matrix(files_10x$matrix.mtx[!(rownames(files_10x$matrix.mtx) %in%
                                                   paste0("gRNA_", annotation$oligo_name, "_gene")),])

In [ ]:
totalGeneCounts <- data.frame(rowSums(matrix_regular))
totalGeneCounts$gene <- rownames(totalGeneCounts)
colnames(totalGeneCounts) <- c("count", "gene")
totalGeneCounts <- totalGeneCounts[totalGeneCounts$count > 0,]
write.csv(totalGeneCounts, sfile(csv, "total_gene_count.csv"))

In [ ]:
# Create data frame for gRNA ranking per cell
gRNA_rank <- data.frame(rank=1:20)

for (cell in colnames(matrix_gRNA)) {
    buffer <- data.frame(count=head(sort(as.vector(matrix_gRNA[,cell]),
                                                  decreasing = TRUE), 20))
    colnames(buffer) <- c(cell)

    gRNA_rank <- cbind(gRNA_rank, buffer)
}

rownames(gRNA_rank) <- gRNA_rank$rank
gRNA_rank$rank <- NULL

In [ ]:
# gRNA rank stats
gRNA_rowStats <- data.frame(rank=1:20,
                            sum=as.vector(rowSums(gRNA_rank)),
                            mean=as.vector(rowMeans(gRNA_rank)),
                            std=rowSds(as.matrix(gRNA_rank)),
                            median=as.vector(rowMedians(as.matrix(gRNA_rank))))

In [ ]:
gRNA_threshold <- 3
ntotal <- length(colnames(gRNA_rank))

# Cell need more than 1 read for a gRNA to be detected as gRNA cell
unassigned <- gRNA_rank[,gRNA_rank[1,] <= gRNA_threshold]
message(paste0("unassigned: ", length(colnames(unassigned)) / ntotal * 100,
               " (", length(colnames(unassigned)), ")"))

assigned <- gRNA_rank[,gRNA_rank[1,] > gRNA_threshold]
singlets <- assigned[,assigned[2,] <= gRNA_threshold]
multiplets <- assigned[,assigned[2,] > gRNA_threshold]

In [ ]:
matrix_gRNA <- matrix_gRNA - gRNA_threshold
matrix_gRNA[matrix_gRNA < 0] <- 0
matrix_gRNA_singlets <- matrix_gRNA[,colSums(matrix_gRNA >= 1) == 1]

In [ ]:
# Save data
saveRDS(matrix_gRNA, sfile(RData, "matrix_gRNA.RDS"))
saveRDS(matrix_regular, sfile(RData, "matrix_regular.RDS"))
saveRDS(matrix_gRNA_singlets, sfile(RData, "matrix_gRNA_singlets.RDS"))

In [ ]:
cell_gRNA_tab <- convertTo_cellgRNA_table(matrix_gRNA, threshold = 1)
cell_gRNA_tab$nGene <- colSums(matrix_regular[,cell_gRNA_tab$cell_barcode] >= 1)
cell_gRNA_tab$nUMI <- colSums(matrix_regular[,cell_gRNA_tab$cell_barcode])
cell_gRNA_singlets_tab <- convertTo_cellgRNA_table(matrix_gRNA_singlets, threshold = 1)
cell_gRNA_singlets_tab$gRNA_group <- sapply(cell_gRNA_singlets_tab$gRNA, function(x){
    annotation[unlist(strsplit(unlist(strsplit(x, '_gene', fixed = TRUE)),
                               "gRNA_", fixed=TRUE))[2] == annotation$oligo_name, "group"]
})

In [ ]:
write.csv(cell_gRNA_tab, file=sfile(csv, "cell_gRNA_tab.csv"))
write.csv(cell_gRNA_singlets_tab, file=sfile(csv, "cell_gRNA_singlets_tab.csv"))

In [ ]:
matrix_regular_gRNASinglets <- matrix_regular[,cell_gRNA_singlets_tab$cell_barcode]
saveRDS(matrix_regular_gRNASinglets, file=sfile(RData, "matrix_regular_gRNASinglets.RDS"))

In [ ]:

percent_unique_gRNA <- vector()
percent_multiple_gRNA <- vector()
ncells <- vector()

thresholds <- c(125, 250, 500, 1000, 2000, 3000, 4000, 5000)

for(i in thresholds){
    cells <- cell_gRNA_tab$cell_barcode[cell_gRNA_tab$nGene >= i]
    ncells <- c(ncells, length(cells))

    percent_unique_gRNA <- c(percent_unique_gRNA,
        sum(cell_gRNA_tab$cell_barcode[cell_gRNA_tab$ngRNA == 1] %in% cells) / length(cells))
    percent_multiple_gRNA <- c(percent_multiple_gRNA,
        sum(cell_gRNA_tab$cell_barcode[cell_gRNA_tab$ngRNA > 1] %in% cells) / length(cells))
}
percent_not_assigned_gRNA <- 1 - (percent_unique_gRNA + percent_multiple_gRNA)

library(reshape2)
quality_genes_gRNA_melt <- melt(data.frame(threshold = factor(thresholds, levels=thresholds),
           singlets = percent_unique_gRNA,
           impurities = percent_multiple_gRNA,
           not_assigned  = percent_not_assigned_gRNA), vars.id="threshold")
quality_genes_gRNA_melt$variable <- factor(quality_genes_gRNA_melt$variable,
                                           levels=c("not_assigned", "singlets", "impurities"))


plot <- ggplot(quality_genes_gRNA_melt, aes(x=threshold, y=value*100, fill=variable)) +
    geom_bar(stat="identity") +
    scale_fill_manual(values = c("darkblue", "peru", "darkgreen")) +
    labs(y="Percent of all cells", x="Genes covered per cell", fill="metric") +
    scale_y_continuous(limits=c(0, 100)) +
    theme(legend.position = c(.7,.85))
plot

pdf(sfile(pdf, "nGenes_gRNA_Assigned.pdf"))
plot
dev.off()

In [ ]:
rownames(cell_gRNA_tab) <- cell_gRNA_tab$cell_barcode
cell_gRNA_tab$gRNA_group <- NA
cell_gRNA_tab[cell_gRNA_singlets_tab$cell_barcode, "gRNA_group"] <- as.vector(cell_gRNA_singlets_tab$gRNA_group)
cell_gRNA_tab$gRNA_version <- NA
cell_gRNA_tab[cell_gRNA_singlets_tab$cell_barcode, "gRNA_version"] <-
    sapply(cell_gRNA_tab[cell_gRNA_singlets_tab$cell_barcode, "gRNA"],
            function(x) return(unlist(strsplit(x, "_"))[2])
    )

In [ ]:
# Export assignmet distribution
gRNA_denstity <- data.frame(gRNAs= c(0:20),
                            counts = c(sapply(0:20, function(x) nrow(cell_gRNA_tab[cell_gRNA_tab$ngRNA == x,]))))
write.csv(gRNA_denstity, file=sfile(csv, "assiged_gRNAs.csv"))

In [ ]:
assignDistribution <- data.frame(groups = c("no gRNA", "single gRNA", "multiple gRNA"), 
                                counts = c(nrow(cell_gRNA_tab[cell_gRNA_tab$ngRNA == 0,]),
    nrow(cell_gRNA_tab[cell_gRNA_tab$ngRNA == 1,]),
    nrow(cell_gRNA_tab[cell_gRNA_tab$ngRNA > 1,])))

In [ ]:
# Plot assignment piechart
percent <- function(x) {
    return(paste0(round(x * 100, 1), "%"))
}

plot <- ggplot(assignDistribution, aes(x="", y=counts, fill=groups))+
    geom_bar(width = 1, stat = "identity") + coord_polar("y", start=0) + theme_minimal()+
    labs(x = NULL, y = NULL, fill = NULL, title = "10X + Primer") +
    theme(axis.title.x = element_blank(),
          axis.title.y = element_blank(),
          panel.border = element_blank(),
          plot.title = element_text(hjust = 0.5,size=20, color = "black"),
          panel.grid=element_blank(),
          axis.ticks = element_blank()) +
    theme(axis.text.x=element_blank()) +
    geom_text(aes(label = percent(counts/sum(counts))),
              position = position_stack(vjust = 0.5),
              size=5) +
    scale_fill_manual(values=c("#9ecae2", "#deebf7", "#3281bd"))


plot

pdf(sfile(pdf, "gRNA_assigned_distribution_PieChart.pdf"), useDingbats = FALSE, width=7, height= 7)
plot
dev.off()

In [ ]:
seu.singlets <- CreateSeuratObject(counts = matrix_regular_gRNASinglets, project = analysis)
seu.singlets@meta.data[cell_gRNA_singlets_tab$cell_barcode, "gRNA_group"] <- cell_gRNA_singlets_tab$gRNA_group

In [ ]:
#Add gRNA_Information to seurat object
seu.singlets@meta.data$gRNA <- NA
seu.singlets@meta.data[cell_gRNA_singlets_tab$cell_barcode, "gRNA"] <-
    cell_gRNA_singlets_tab$gRNA
seu.singlets@meta.data$gRNA_group <- NA
seu.singlets@meta.data[cell_gRNA_singlets_tab$cell_barcode, "gRNA_group"] <-
    as.vector(cell_gRNA_singlets_tab$gRNA_group)

In [ ]:
# Calculating percentage of mitochondrial genes
mt.genes.idx <- grep('mt-', rownames(seu.singlets)) 
mt.genes.per <- Matrix::colSums(seu.singlets[mt.genes.idx,])/Matrix::colSums(seu.singlets)
seu.singlets <- AddMetaData(seu.singlets, metadata=mt.genes.per, col.name='mito.percent')

In [ ]:
# Isolate cells
seu.singlets <- subset(seu.singlets, subset = nFeature_RNA > 100 & mito.percent < 0.1)

# Remove MITO genes
seu.singlets <- seu.singlets[-mt.genes.idx, ]

In [ ]:
freq_gRNA <- data.frame(table(seu.singlets@meta.data$gRNA))
keep_cells <- rownames(seu.singlets@meta.data)[seu.singlets@meta.data$gRNA %in%
                                               as.vector(freq_gRNA[freq_gRNA$Freq >= 10, "Var1"])]

In [ ]:
seu.singlets <- subset(seu.singlets, cells = keep_cells)

In [ ]:
seu.singlets <- ScaleData(object = seu.singlets, verbose = FALSE)

In [ ]:
seu.singlets <- NormalizeData(seu.singlets, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
seu.singlets <- FindVariableFeatures(seu.singlets, selection.method = "vst", nfeatures = 2000)

In [ ]:
seu.singlets <- RunPCA(object = seu.singlets, npcs = 30, verbose = FALSE)

In [ ]:
seu.singlets <- RunUMAP(object = seu.singlets,
                                   reduction = "pca", 
                                   dims = 1:30, verbose = FALSE)

In [ ]:
seu.singlets <- FindNeighbors(seu.singlets, dims = 1:10)
seu.singlets <- FindClusters(seu.singlets, resolution = 0.5)

In [ ]:
cc.genes <- readLines(con = file.path(external, "regev_lab_cell_cycle_genes.txt"))
s.genes <- stri_trans_totitle(cc.genes[1:43])
g2m.genes <- stri_trans_totitle(cc.genes[44:97])

seu.singlets <- CellCycleScoring(object = seu.singlets,
                                 s.features = s.genes,
                                 g2m.features = g2m.genes,
                                 set.ident = TRUE)

In [ ]:
Freq_phase_data <- data.frame(table(seu.singlets@meta.data[which(
                                seu.singlets@meta.data$gRNA_group == "control"), "Phase"]),
           group=rep("control",3))

for(gR_group in names(table(seu.singlets@meta.data$gRNA_group))){
    if(gR_group == "control") next
    current_table <- table(seu.singlets@meta.data[which(
                                seu.singlets@meta.data$gRNA_group == gR_group),"Phase"])

    current_group <- data.frame(current_table,
                               group=rep(gR_group, length(current_table)))

    Freq_phase_data <- rbind(Freq_phase_data, current_group)
}


p1 <- ggplot(Freq_phase_data, aes(group, Freq, fill=Var1)) +
geom_bar(position = position_fill(reverse = TRUE), stat="Identity", color="black") +
labs(x="", y="", fill="Cluster") +
theme(axis.text.x = element_text(angle=90, hjust = 1), axis.text.y = element_text(size=4))

p2 <- ggplot(Freq_phase_data, aes(group, Freq, fill=Var1)) +
geom_bar(position = position_stack(reverse = TRUE), stat="Identity", color="black") +
labs(x="", y="", fill="Phase") +
theme(axis.text.x = element_text(angle=90, hjust = 1), axis.text.y = element_text(size=4))

p2 + p1

pdf(sfile(pdf, "Ratio_gRNA_groups_Phase.pdf"))
grid.arrange(p2, p1, ncol=1)
dev.off()

In [ ]:
seu.singlets <- seu.singlets[,seu.singlets@meta.data$Phase == "G1"]

In [ ]:
# Recompute PCA
seu.singlets <- FindVariableFeatures(seu.singlets, selection.method = "vst", nfeatures = 2000)
seu.singlets <- RunPCA(object = seu.singlets, npcs = 30, verbose = FALSE)

# Cluster analysis and UMAP
seu.singlets <- RunUMAP(object = seu.singlets,
                                   reduction = "pca",
                                   dims = 1:30, verbose = FALSE)

# Recompute clustering
seu.singlets <- FindNeighbors(seu.singlets, dims = 1:10, verbose=FALSE)
seu.singlets <- FindClusters(seu.singlets, resolution = 0.5, verbose=FALSE)

In [ ]:
plot1 <- DimPlot(seu.singlets, reduction = "umap", group.by="seurat_clusters", label=FALSE, pt.size = 1) + 
    theme(aspect.ratio=1) + coord_fixed()


pdf(sfile(pdf, "ClusterAnalysis_onlyG1_UMAP_plot.pdf"))
plot1
dev.off()

plot1

In [ ]:
# Load score genes
Th17_genes <- c("Rorc", "Il17a", "Il17f", "Il23r", "Il6ra", "Il22", "Ccr6")

Th1_genes <- c("Tbx21", "Ifng", "Ccr5", "Ccl5", "Cxcr3", "Il12rb1")

Treg_genes <- c("Lag3", "Havcr2", "Pdcd1", "Ctla4", "Itga2",
                "Foxp3", "Maf", "Irf1", "Batf",
                "Il12rb2", "Tnfrsf18", "Nfil3", "Ahr", "Rora","Il2ra", "Icos")

In [ ]:
seu.singlets <- AddModuleScore(seu.singlets,
                                          ctrl = 50,
                               name = 'Th17_Sig',
                               features = list(Th17_genes))

seu.singlets <- AddModuleScore(seu.singlets,
                                          ctrl = 50,
                               name = 'Th1_Sig',
                               features = list(Th1_genes))

seu.singlets <- AddModuleScore(seu.singlets,
                                          ctrl = 50,
                               name = 'Treg_Sig',
                               features = list(Treg_genes))


In [ ]:
plot1 <- VlnPlot(seu.singlets, features = c("Th17_Sig1"), sort = FALSE, pt.size = 0) +
            stat_summary(fun = median, geom='point', size = 25, colour = "black", shape = 95) +
            NoLegend()

plot2 <- VlnPlot(seu.singlets, features = c("Th1_Sig1"), sort = FALSE, pt.size = 0) +
            stat_summary(fun = median, geom='point', size = 25, colour = "black", shape = 95) +
            NoLegend()
plot3 <- VlnPlot(seu.singlets, features = c("Treg_Sig1"), sort = FALSE, pt.size = 0) +
            stat_summary(fun = median, geom='point', size = 25, colour = "black", shape = 95) +
            NoLegend()

plot1
plot2
plot3


pdf(sfile(pdf, "Signature_scores.pdf"), width=6, height=18)
plot1 + plot2 + plot3
dev.off()

In [ ]:
plot <- FeaturePlot(seu.singlets, features = c("Th17_Sig1", "Th1_Sig1", "Treg_Sig1"), pt.size = 0.5)

plot

pdf(sfile(pdf, "FeaturePlot_allSignatureScores_allCells.pdf"))
plot
dev.off()

In [ ]:
Th17_barcodes <- rownames(seu.singlets@meta.data[seu.singlets@meta.data$seurat_clusters %in% c(2) &
                                                 seu.singlets@meta.data$gRNA_group == "control",])
Th1_barcodes <- rownames(seu.singlets@meta.data[seu.singlets@meta.data$seurat_clusters %in% c(3) &
                                                seu.singlets@meta.data$gRNA_group == "control",])
Treg_barcodes <- rownames(seu.singlets@meta.data[seu.singlets@meta.data$seurat_clusters %in% c(4) &
                                                 seu.singlets@meta.data$gRNA_group == "control",])

In [ ]:
groups <- unique(seu.singlets@meta.data$gRNA_group)

In [ ]:
get_UMAP_Median_by_cells <- function(seuObj, cells, group) {
    seuSubset <- subset(seuObj, cells = cells)
    umapSubset <- data.frame(Embeddings(seuSubset, reduction = "umap"))
    return(data.frame(group = group,
                      umap1 = median(umapSubset$UMAP_1),
                      umap2 = median(umapSubset$UMAP_2),
                      counts = length(rownames(seuSubset@meta.data))))
}

In [ ]:
getControlGroupLines <- function(umapMedians) {
    centerControl <- umapMedians[umapMedians$group == "control",]

    controlCoords <- data.frame()
    transDiffSubet <- subset(umapMedians, grepl("_control$", group))

    for (group in transDiffSubet$group) {
        targetControl <- umapMedians[umapMedians$group == group,]

        controlCoords <- rbind(controlCoords, data.frame(x1=centerControl$umap1,
                                                         y1=centerControl$umap2,
                                                         x2=targetControl$umap1,
                                                         y2=targetControl$umap2))
    }
    return(controlCoords)
}

In [ ]:
UMAP_Medians <- data.frame()

for (g in 1:length(groups)) {
    if (groups[g] != "control") {
        UMAP_Medians <- rbind(UMAP_Medians, get_UMAP_Median_by_cells(seuObj = seu.singlets,
                           cells = rownames(seu.singlets@meta.data[
                              seu.singlets@meta.data$gRNA_group == groups[g],]),
                           group = groups[g]))
    }
}

# Add UMAP Median for control subsets
UMAP_Medians <- rbind(UMAP_Medians, get_UMAP_Median_by_cells(seuObj = seu.singlets,
                             cells = Th17_barcodes,
                             group = "Th17_control"))


UMAP_Medians <- rbind(UMAP_Medians, get_UMAP_Median_by_cells(seuObj = seu.singlets,
                             cells = Th1_barcodes,
                             group = "Th1_control"))


UMAP_Medians <- rbind(UMAP_Medians, get_UMAP_Median_by_cells(seuObj = seu.singlets,
                             cells = Treg_barcodes,
                             group = "Treg_control"))


# All control subset
UMAP_Medians <- rbind(UMAP_Medians, get_UMAP_Median_by_cells(seuObj = seu.singlets,
                             cells = rownames(seu.singlets@meta.data[seu.singlets@meta.data$gRNA_group == "control",]),
                             group = "control"))



library(ggrepel)

ggtheme <- theme(plot.title = element_text(face = 'bold',size = 18, hjust = 0.5),
    axis.title = element_text(size=8),
    axis.text = element_text(size=8),
    legend.position="none")


p1 <- ggplot(UMAP_Medians, aes(umap1, umap2)) +
    geom_segment(data=getControlGroupLines(UMAP_Medians),
                 aes(x = x1, y = y1, xend = x2, yend = y2),
                 color="#cccccc",
                 lineend="round", linejoin="round",
                 arrow=arrow(length = unit(0.1, "inches")),
                 size=1) +
    geom_point(aes(size=counts), color="black") +
    geom_point(data = subset(UMAP_Medians, grepl("_control$", group)),
               color="blue",
               aes(umap1, umap2, size=counts)) +
    geom_point(data = subset(UMAP_Medians, grepl("^control$", group)),
               color="red",
               aes(umap1, umap2, size=counts)) +
    labs(x="UMAP_1", y="UMAP_2", title="UMAP Medians per gRNA group") +
    geom_label_repel(aes(label=group), size=2.5, alpha=.7, max.overlaps = 15) +
    theme_classic() + theme(aspect.ratio=1) + coord_fixed()

p1

pdf(sfile(pdf, "UMAP_median_bygRNA_group.pdf"), useDingbats = FALSE, width=6, height=6)
p1
dev.off()


In [ ]:
groups <- unique(seu.singlets@meta.data$gRNA_group)

pdf(sfile(pdf, "gRNAGroups_UMAP_Highlight.pdf"), useDingbats = FALSE, height=5, width=5)

for(g in groups){
    pos <- rownames(seu.singlets@meta.data[seu.singlets@meta.data$gRNA_group == g,])

    print(DimPlot(seu.singlets, reduction = "umap", cells.highlight = pos, sizes.highlight=1.5) +
    theme(aspect.ratio=1,
           legend.position = "none",
         axis.ticks = element_blank(),
         axis.text = element_blank()) + labs(title=g))
}
dev.off()


In [ ]:
saveRDS(seu.singlets, sfile(RData, "seurat_Singlets.rds"))

In [ ]:
# Seurat object is then converted to h5ad object using SeuratDisk
# See https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html